### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [2]:
!pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.5/763.5 KB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.

### Load data and model

In [3]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [4]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [5]:
MAX_LENGTH = 128
# описание параметров: https://huggingface.co/docs/transformers/main_classes/tokenizer
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'],     #токенизируемый текст1
        examples['text2'],     #токенизируемый текст2
        padding='max_length',  #padding до макс. длины
        max_length=MAX_LENGTH, #максимальная длина предложения
        truncation=True        #усечение до максимальной длине, указанной в max_length
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [6]:
#repr - печатное представление объекта
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [7]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, 
    batch_size=1, 
    shuffle=False, 
    collate_fn=transformers.default_data_collator #the collate_fn argument is used to collate lists of samples into batches
)

In [8]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
    predicted = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        token_type_ids=batch['token_type_ids']
    )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
# <A whole lot of YOUR CODE HERE>
from tqdm import tqdm
from datasets import load_metric
from IPython.display import clear_output

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
val_loader = torch.utils.data.DataLoader(
    val_set, 
    batch_size=30, 
    shuffle=False,
    num_workers=4,                                 #num_workers – how many subprocesses to use for data loading
    collate_fn=transformers.default_data_collator  #the collate_fn argument is used to collate lists of samples into batches
)

true_y = torch.empty(0).to(device)
pred_y = torch.empty(0).to(device)

metric = load_metric("accuracy")

for batch in tqdm(val_loader):

    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
    
    labels = batch['labels'].to(device)
    predictions = torch.argmax(torch.softmax(predicted.logits, dim=1), dim=-1)
    
    true_y = torch.cat((true_y, labels))
    pred_y = torch.cat((pred_y, predictions))
    acc = metric.compute(predictions=predictions, references=labels)
    
    clear_output(wait=True)
    print(acc)
    
accuracy = metric.compute(predictions=pred_y, references=true_y)['accuracy'] #<Validation accuracy, between 0 and 1>

100%|██████████| 1348/1348 [01:18<00:00, 17.22it/s]

{'accuracy': 0.8}


In [ ]:
assert 0.9 < accuracy < 0.91
accuracy

0.9083848627256987

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
# <A whole lot of your code goes here>

In [9]:
dataloaders = {}
for i in ['train','validation']:
    dataloaders[i] = torch.utils.data.DataLoader(
                            qqp_preprocessed[i], 
                            batch_size=30, 
                            shuffle=False,
                            #num_workers=2,
                            collate_fn=transformers.default_data_collator
                        )

criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
import time
import copy

# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
def train_model(model, dataloaders, criterion, optimizer, num_epochs=1):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0

            # Iterate over data.
            for batch in tqdm(dataloaders[phase]):

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    predicted = model(
                                    input_ids=batch['input_ids'].to(device),
                                    attention_mask=batch['attention_mask'].to(device),
                                    token_type_ids=batch['token_type_ids'].to(device)
                                )
                    outputs = torch.softmax(predicted.logits, dim=1)
                    labels = batch['labels'].to(device)

                    loss = criterion(outputs, labels)

                    preds = torch.max(outputs, 1).indices

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * labels.size()[0]
                running_acc += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'validation':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [14]:
model = transformers.AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').to(device)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [ ]:
train_model(model, dataloaders, criterion, opt, num_epochs=25)

Epoch 0/24
----------


100%|██████████| 12129/12129 [39:28<00:00,  5.12it/s]


train Loss: 0.6985 Acc: 0.4778


100%|██████████| 1348/1348 [01:29<00:00, 15.09it/s]


validation Loss: 0.6988 Acc: 0.4842

Epoch 1/24
----------


...

train Loss: 0.6986 Acc: 0.4772


100%|██████████| 1348/1348 [01:29<00:00, 15.10it/s]


validation Loss: 0.6988 Acc: 0.4842

Epoch 4/24
----------


 33%|███▎      | 3965/12129 [12:54<26:33,  5.12it/s]


KeyboardInterrupt: ignored

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [170]:
import numpy as np

# сбор текстов из train
print('сбор текстов из train')
train_text_list = []
for i in tqdm(qqp['train']):
  train_text_list.append(i['text1'])
  train_text_list.append(i['text2'])
print('\nlen:', len(train_text_list))
train_text_list = list(set(train_text_list))
print('len after dedublication:', len(train_text_list), '\n')

сбор текстов из train


100%|██████████| 363846/363846 [00:17<00:00, 21358.25it/s]



len: 727692
len after dedublication: 493874 



In [180]:
def k_duplicate_questions(my_text: str, all_text: list, example_count: int, search_limit=10000):
  # скоринг входного предложения с каждым предложение из train
  print('скоринг')
  pred_list = []
  s = 0

  for text_i in tqdm(train_text_list):
    dict_i = {'text1': my_text, 'text2': text_i[0], 'label': None}
    batch_i = preprocess_function(dict_i)

    for k in batch_i:
      if k!='label':
        batch_i[k] = torch.tensor(batch_i[k])
        batch_i[k] = torch.unsqueeze(batch_i[k], dim=0)

    with torch.no_grad():
      predicted = model(
        input_ids=batch_i['input_ids'].to(device),
        attention_mask=batch_i['attention_mask'].to(device),
        token_type_ids=batch_i['token_type_ids'].to(device)
      )

    output_i = torch.softmax(predicted.logits, dim=1)
    pred_i = output_i[0][1].item()
    pred_list.append(pred_i)

    s+=1
    if s>search_limit: break

  # поиск наиболее близких предолжений
  print(f'\nваше предложение: {my_text}\n')
  print(f'\n{example_count} наиболее близких предложений:')
  index_list = []

  count = 0
  for max_val_i in set(sorted(pred_list, reverse=True)):
    index_i = np.where(np.array(pred_list) == max_val_i)[0]
    count += len(index_i)
    index_list.extend(index_i.tolist())
    if count>=example_count: break

  for x,y in zip( np.array(train_text_list)[index_list[:example_count]],
                  np.array(pred_list)[index_list[:example_count]] ):
    print(x, f'({y.round(5)})')
  
  return None

In [181]:
k_duplicate_questions(my_text='How is the life of a math student? Could you describe your own experiences?',
                      all_text=train_text_list,
                      example_count=5
                     )

скоринг


  2%|▏         | 10000/493874 [03:47<3:03:29, 43.95it/s]



ваше предложение: How is the life of a math student? Could you describe your own experiences?


5 наиболее близких предложений:
Physical Chemistry: What is molecular vibration? (0.5158)
Pokemon GO looks like the dumbest game ever; how is it so popular? (0.5158)
PHP is good or bad? (0.5158)
Public Speaking: I have to give one humorous speech for almost 5-7 minutes. What are some suggestions for topics? (0.5158)
Preparation of trans uranic elements? (0.5158)


In [182]:
k_duplicate_questions(my_text='What is success in work?',
                      all_text=train_text_list,
                      example_count=5,
                      search_limit=5000
                     )

скоринг


  1%|          | 5000/493874 [01:53<3:05:12, 43.99it/s]



ваше предложение: What is success in work?


5 наиболее близких предложений:
Libra female and scorpio male as best friends or best friends with benefits? (0.5497)
Valuelabs aptitude pattern? (0.53445)
Video Game Artificial Intelligence: Is the GPU ever involved in video game AI? (0.53445)
Can the rolling wheels of a car generate enough electricity so the car itself can be completely energy independent? (0.53046)
Can a noncitizen immigrant donate to a political campaign since spending money constitutes free speech? (0.53046)


In [183]:
k_duplicate_questions(my_text='why is the weather so bad',
                      all_text=train_text_list,
                      example_count=5,
                      search_limit=1000
                     )

скоринг


  0%|          | 1000/493874 [00:22<3:07:18, 43.86it/s]



ваше предложение: why is the weather so bad


5 наиболее близких предложений:
Just started to take antidepressant as part of my treatment, what should I expect now? (0.57026)
From where can I order a diary with full pages for Sundays also? (0.56808)
Physical Chemistry: What is molecular vibration? (0.54427)
Pokemon GO looks like the dumbest game ever; how is it so popular? (0.54427)
PHP is good or bad? (0.54427)


__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.